In [15]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

#imdb data collection
url = "https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

#creating empty list to append values later
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []

movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})

#Looping the process and appending data into empty list
for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)
    
    #removing unnecessary bracket using 'replace'
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)
    
    #removing unnecessary space using 'replace'
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    #removing \n problems using 'replace'
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
    metascore.append(meta)
    #gross and votes have same attributes, problem solved using creation of a common variable and then used indexing
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    vote = value[0].text
    votes.append(vote)
    
    grosses = value[1].text if len(value) >1 else '*****'
    gross.append(grosses)

#creating DataFrame and storing it as CSV for further analysis
movie_DF = pd.DataFrame({'Movie title': movie_name, 'Year released': year, 'Duration': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Total gross': gross})

movie_DF.to_csv("Top_100_IMDB_Movies.csv", index=False)